# Caso Práctico Colaborativo - Bloque 2
## Scouting y Análisis Táctico con Datos
### SOLUCIÓN COMPLETA

**Equipo:** Analistas de Scouting Pro  
**Integrantes:** [Ejemplo de equipo]  
**Fecha:** Agosto 2025

---

## Contexto del Problema

Como analistas de datos especializados en scouting, hemos sido contratados por una academia de fútbol profesional para crear un sistema de búsqueda de talentos basado en estadísticas. Nuestro objetivo es identificar patrones en el rendimiento de jugadores jóvenes para optimizar las decisiones de reclutamiento.

## Objetivos de Aprendizaje:
- Aplicar técnicas de exploración de datos en contextos deportivos
- Crear visualizaciones informativas con seaborn y matplotlib
- Realizar análisis estadístico descriptivo avanzado
- Identificar patrones y tendencias en datos deportivos
- Comunicar hallazgos técnicos a audiencias no técnicas

In [ ]:
# Importaciones necesarias para análisis de datos deportivos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración básica de matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Intentar importar seaborn si está disponible
try:
    import seaborn as sns
    sns.set_theme(style="whitegrid", palette="Set2")
    SEABORN_AVAILABLE = True
    print("✅ Seaborn disponible - gráficos avanzados habilitados")
except ImportError:
    SEABORN_AVAILABLE = False
    print("⚠️  Seaborn no disponible - usando matplotlib básico")

try:
    from sklearn.linear_model import LinearRegression
    SKLEARN_AVAILABLE = True
    print("✅ Scikit-learn disponible - modelos predictivos habilitados")
except ImportError:
    SKLEARN_AVAILABLE = False
    print("⚠️  Scikit-learn no disponible - usando análisis estadístico básico")

print("¡Herramientas de análisis avanzado listas!")
print(f"Pandas: {pd.__version__} | NumPy: {np.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
if SEABORN_AVAILABLE:
    print(f"Seaborn: {sns.__version__}")

In [ ]:
# Cargar los datasets del sistema de scouting
print("📊 Cargando datasets del sistema de scouting...")

# Dataset principal de jugadores
df_jugadores = pd.read_csv('../datasets/jugadores_liga_juvenil.csv')
print(f"✅ Jugadores cargados: {len(df_jugadores)} registros")

# Dataset de detalles de partidos
df_partidos = pd.read_csv('../datasets/partidos_detalle.csv')
print(f"✅ Partidos cargados: {len(df_partidos)} registros")

# Dataset de eventos durante los partidos
df_eventos = pd.read_csv('../datasets/eventos_partidos.csv')
print(f"✅ Eventos cargados: {len(df_eventos)} registros")

print(f"\n🎯 Sistema de scouting inicializado correctamente")
print(f"Total de datos disponibles: {len(df_jugadores)} jugadores, {len(df_partidos)} partidos, {len(df_eventos)} eventos")

## 1.2 Análisis Exploratorio de Datos (15 puntos)

In [ ]:
def analizar_dataset(df, nombre_dataset):
    """
    Función para análisis exploratorio estandarizado
    Proporciona un resumen completo de la estructura y calidad de los datos
    """
    print(f"\n=== ANÁLISIS DE {nombre_dataset.upper()} ===")
    print(f"📏 Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
    print(f"📋 Columnas: {list(df.columns)}")
    
    print(f"\n🔍 Valores faltantes por columna:")
    valores_faltantes = df.isnull().sum()
    if valores_faltantes.sum() == 0:
        print("   ✅ ¡Excelente! No hay valores faltantes")
    else:
        for col, missing in valores_faltantes[valores_faltantes > 0].items():
            porcentaje = (missing / len(df)) * 100
            print(f"   ⚠️  {col}: {missing} ({porcentaje:.1f}%)")
    
    print(f"\n📊 Estadísticas descriptivas de variables numéricas:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(df[numeric_cols].describe().round(2))
    
    print(f"\n🏷️ Variables categóricas y sus valores únicos:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        unique_values = df[col].nunique()
        print(f"   • {col}: {unique_values} valores únicos")
        if unique_values <= 10:
            print(f"     Valores: {list(df[col].unique())}")
    
    return df.info()

# Aplicar análisis a cada dataset
analizar_dataset(df_jugadores, "jugadores liga juvenil")

In [ ]:
analizar_dataset(df_partidos, "partidos detalle")

In [ ]:
analizar_dataset(df_eventos, "eventos partidos")

In [ ]:
# Análisis específico de distribuciones clave para scouting
print("🎯 ANÁLISIS ESPECÍFICO PARA SCOUTING")
print("=" * 50)

# Distribución de edades por posición
print("\n👥 DISTRIBUCIÓN DE EDADES POR POSICIÓN:")
edad_por_posicion = df_jugadores.groupby('posicion')['edad'].agg(['mean', 'min', 'max', 'std']).round(1)
print(edad_por_posicion)

# Distribución de goles por competición
print("\n⚽ DISTRIBUCIÓN DE GOLES POR COMPETICIÓN:")
goles_por_competicion = df_jugadores.groupby('torneo')['goles'].agg(['mean', 'sum', 'max']).round(1)
print(goles_por_competicion)

# Análisis de valores de mercado
print("\n💰 ANÁLISIS DE VALORES DE MERCADO:")
print(f"Valor promedio: ${df_jugadores['valor_mercado_estimado'].mean():,.0f}")
print(f"Valor mediano: ${df_jugadores['valor_mercado_estimado'].median():,.0f}")
print(f"Rango: ${df_jugadores['valor_mercado_estimado'].min():,.0f} - ${df_jugadores['valor_mercado_estimado'].max():,.0f}")

# Identificar valores atípicos en rendimiento
print("\n🎯 ANÁLISIS DE VALORES ATÍPICOS:")
q1_goles = df_jugadores['goles'].quantile(0.25)
q3_goles = df_jugadores['goles'].quantile(0.75)
iqr_goles = q3_goles - q1_goles
limite_superior = q3_goles + 1.5 * iqr_goles

outliers_goles = df_jugadores[df_jugadores['goles'] > limite_superior]
print(f"Jugadores con rendimiento goleador excepcional: {len(outliers_goles)}")
if len(outliers_goles) > 0:
    print("Top goleadores atípicos:")
    for idx, jugador in outliers_goles.nlargest(3, 'goles').iterrows():
        print(f"  • {jugador['nombre']} ({jugador['edad']} años): {jugador['goles']} goles - {jugador['posicion']}")

## 1.3 Limpieza de Datos (10 puntos)

In [ ]:
# Limpieza y preparación de datos para análisis de scouting
print("🧹 PROCESO DE LIMPIEZA DE DATOS")
print("=" * 40)

# 1. Verificar y limpiar valores faltantes
print("\n1️⃣ Verificando valores faltantes...")
total_faltantes = df_jugadores.isnull().sum().sum()
if total_faltantes == 0:
    print("   ✅ No se encontraron valores faltantes")
else:
    print(f"   ⚠️  Se encontraron {total_faltantes} valores faltantes")
    # Aquí se implementaría la lógica de manejo según el tipo de columna

# 2. Normalizar nombres de equipos y posiciones
print("\n2️⃣ Normalizando categorías...")
# Verificar consistencia en nombres de posiciones
posiciones_unicas = df_jugadores['posicion'].unique()
print(f"   Posiciones encontradas: {list(posiciones_unicas)}")

# Verificar consistencia en nombres de torneos
torneos_unicos = df_jugadores['torneo'].unique()
print(f"   Torneos encontrados: {list(torneos_unicos)}")

# 3. Crear variables derivadas para análisis de scouting
print("\n3️⃣ Creando variables derivadas para scouting...")

# Variables de eficiencia
df_jugadores['goles_por_partido'] = df_jugadores['goles'] / df_jugadores['partidos_jugados']
df_jugadores['asistencias_por_partido'] = df_jugadores['asistencias'] / df_jugadores['partidos_jugados']
df_jugadores['minutos_por_partido'] = df_jugadores['minutos_jugados'] / df_jugadores['partidos_jugados']

# Eficiencia de tiro (solo para jugadores con tiros)
df_jugadores['eficiencia_tiros'] = np.where(
    df_jugadores['tiros_al_arco'] > 0,
    df_jugadores['goles'] / df_jugadores['tiros_al_arco'] * 100,
    0
)

# Índice de contribución ofensiva
df_jugadores['contribucion_ofensiva'] = df_jugadores['goles'] + (df_jugadores['asistencias'] * 0.7)

# Categorización por edad
df_jugadores['categoria_edad'] = pd.cut(
    df_jugadores['edad'], 
    bins=[15, 16, 17, 18, 19], 
    labels=['16 años', '17 años', '18 años', '19+ años'],
    include_lowest=True
)

# Índice de disciplina (menos tarjetas es mejor)
df_jugadores['indice_disciplina'] = np.where(
    df_jugadores['partidos_jugados'] > 0,
    100 - ((df_jugadores['tarjetas_amarillas'] * 2 + df_jugadores['tarjetas_rojas'] * 10) / 
           df_jugadores['partidos_jugados'] * 10),
    100
)
df_jugadores['indice_disciplina'] = df_jugadores['indice_disciplina'].clip(0, 100)

print(f"   ✅ Variables creadas: goles_por_partido, asistencias_por_partido, eficiencia_tiros")
print(f"   ✅ Variables creadas: contribucion_ofensiva, categoria_edad, indice_disciplina")

# 4. Validar rangos lógicos
print("\n4️⃣ Validando rangos lógicos...")

# Verificar rangos de edad
edades_invalidas = df_jugadores[(df_jugadores['edad'] < 15) | (df_jugadores['edad'] > 20)]
print(f"   Edades fuera de rango (15-20): {len(edades_invalidas)} jugadores")

# Verificar minutos jugados vs partidos
minutos_invalidos = df_jugadores[df_jugadores['minutos_por_partido'] > 90]
print(f"   Minutos por partido > 90: {len(minutos_invalidos)} casos")

# Verificar precisión de pases
precision_invalida = df_jugadores[(df_jugadores['precision_pases'] < 0) | (df_jugadores['precision_pases'] > 100)]
print(f"   Precisión de pases fuera de rango (0-100%): {len(precision_invalida)} casos")

print(f"\n✅ Proceso de limpieza completado")
print(f"📊 Dataset final: {df_jugadores.shape[0]} jugadores con {df_jugadores.shape[1]} variables")

---
# PARTE 2: ANÁLISIS ESTADÍSTICO AVANZADO (40 puntos)

## 2.1 Estadística Descriptiva por Categorías (15 puntos)

In [ ]:
def analizar_por_posicion(df):
    """
    Análisis comparativo de rendimiento por posición
    Esencial para entender los perfiles de cada posición en scouting
    """
    print("⚽ ANÁLISIS COMPARATIVO POR POSICIÓN")
    print("=" * 45)
    
    # Estadísticas agregadas por posición
    stats_posicion = df.groupby('posicion').agg({
        'goles': ['count', 'mean', 'median', 'std', 'max'],
        'asistencias': ['mean', 'median', 'max'],
        'precision_pases': ['mean', 'std'],
        'edad': ['mean', 'std'],
        'valor_mercado_estimado': ['mean', 'median', 'max'],
        'goles_por_partido': ['mean', 'std'],
        'contribucion_ofensiva': ['mean', 'max'],
        'indice_disciplina': ['mean', 'std']
    }).round(2)
    
    # Aplanar el índice de columnas para mejor visualización
    stats_posicion.columns = ['_'.join(col).strip() for col in stats_posicion.columns.values]
    
    print("\n📊 Estadísticas Completas por Posición:")
    print(stats_posicion)
    
    # Análisis específico por posición
    print("\n🎯 INSIGHTS POR POSICIÓN:")
    for posicion in df['posicion'].unique():
        datos_pos = df[df['posicion'] == posicion]
        print(f"\n{posicion}s ({len(datos_pos)} jugadores):")
        print(f"  • Promedio goles/partido: {datos_pos['goles_por_partido'].mean():.2f}")
        print(f"  • Promedio asistencias/partido: {datos_pos['asistencias_por_partido'].mean():.2f}")
        print(f"  • Precisión promedio pases: {datos_pos['precision_pases'].mean():.1f}%")
        print(f"  • Valor promedio mercado: ${datos_pos['valor_mercado_estimado'].mean():,.0f}")
        print(f"  • Edad promedio: {datos_pos['edad'].mean():.1f} años")
        
        # Jugador destacado por posición
        mejor_jugador = datos_pos.loc[datos_pos['contribucion_ofensiva'].idxmax()]
        print(f"  ⭐ Destacado: {mejor_jugador['nombre']} (Contribución: {mejor_jugador['contribucion_ofensiva']:.1f})")
    
    return stats_posicion

# Ejecutar análisis por posición
estadisticas_posicion = analizar_por_posicion(df_jugadores)

## 3. VISUALIZACIONES Y DASHBOARD (25 puntos)

In [ ]:
# Dashboard principal de análisis de scouting
print("📊 CREANDO DASHBOARD DE ANÁLISIS DE SCOUTING")
print("=" * 50)

# Configurar el dashboard principal
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Dashboard de Scouting - Liga Juvenil MX', fontsize=20, fontweight='bold')

# Gráfico 1: Distribución de goles por posición (con/sin seaborn)
if SEABORN_AVAILABLE:
    sns.boxplot(data=df_jugadores, x='posicion', y='goles', ax=axes[0,0], palette='Set2')
else:
    # Fallback con matplotlib
    posiciones = df_jugadores['posicion'].unique()
    goles_por_pos = [df_jugadores[df_jugadores['posicion'] == pos]['goles'] for pos in posiciones]
    axes[0,0].boxplot(goles_por_pos, labels=posiciones)

axes[0,0].set_title('Distribución de Goles por Posición', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Posición', fontsize=12)
axes[0,0].set_ylabel('Goles Totales', fontsize=12)
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# Añadir estadísticas al gráfico
for i, posicion in enumerate(df_jugadores['posicion'].unique()):
    datos_pos = df_jugadores[df_jugadores['posicion'] == posicion]['goles']
    media = datos_pos.mean()
    axes[0,0].text(i+1, media + 1, f'μ={media:.1f}', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Relación edad vs valor de mercado
if SEABORN_AVAILABLE:
    scatter = sns.scatterplot(data=df_jugadores, x='edad', y='valor_mercado_estimado', 
                             hue='posicion', size='contribucion_ofensiva', 
                             sizes=(50, 200), alpha=0.7, ax=axes[0,1])
else:
    # Fallback con matplotlib - colores por posición
    posiciones = df_jugadores['posicion'].unique()
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
    
    for i, pos in enumerate(posiciones):
        data_pos = df_jugadores[df_jugadores['posicion'] == pos]
        axes[0,1].scatter(data_pos['edad'], data_pos['valor_mercado_estimado'], 
                         c=colors[i % len(colors)], alpha=0.7, label=pos,
                         s=data_pos['contribucion_ofensiva']*5)
    axes[0,1].legend()

axes[0,1].set_title('Edad vs Valor de Mercado', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Edad (años)', fontsize=12)
axes[0,1].set_ylabel('Valor de Mercado (USD)', fontsize=12)
axes[0,1].grid(True, alpha=0.3)

# Formatear eje Y con formato de moneda
axes[0,1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Gráfico 3: Heatmap de correlaciones (con/sin seaborn)
vars_correlacion = ['goles_por_partido', 'asistencias_por_partido', 'precision_pases', 
                   'contribucion_ofensiva', 'valor_mercado_estimado', 'edad', 'indice_disciplina']
correlation_subset = df_jugadores[vars_correlacion].corr()

if SEABORN_AVAILABLE:
    sns.heatmap(correlation_subset, annot=True, cmap='RdBu_r', center=0, 
                square=True, linewidths=0.5, ax=axes[1,0],
                cbar_kws={"shrink": .8})
else:
    # Fallback con matplotlib
    im = axes[1,0].imshow(correlation_subset.values, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[1,0].set_xticks(range(len(correlation_subset.columns)))
    axes[1,0].set_yticks(range(len(correlation_subset.columns)))
    axes[1,0].set_xticklabels([col[:12] + '...' if len(col) > 15 else col for col in correlation_subset.columns])
    axes[1,0].set_yticklabels([col[:12] + '...' if len(col) > 15 else col for col in correlation_subset.columns])
    
    # Añadir valores de correlación
    for i in range(len(correlation_subset.columns)):
        for j in range(len(correlation_subset.columns)):
            text = axes[1,0].text(j, i, f'{correlation_subset.iloc[i, j]:.2f}',
                                 ha="center", va="center", color="black", fontsize=8)
    
    plt.colorbar(im, ax=axes[1,0], shrink=0.8)

axes[1,0].set_title('Matriz de Correlaciones - Métricas Clave', fontsize=14, fontweight='bold')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].tick_params(axis='y', rotation=0)

# Gráfico 4: Top 10 jugadores por contribución ofensiva
top_contribucion = df_jugadores.nlargest(10, 'contribucion_ofensiva')

if SEABORN_AVAILABLE:
    sns.barplot(data=top_contribucion, y='nombre', x='contribucion_ofensiva', 
                palette='viridis', ax=axes[1,1])
else:
    # Fallback con matplotlib
    y_pos = range(len(top_contribucion))
    axes[1,1].barh(y_pos, top_contribucion['contribucion_ofensiva'], color='skyblue')
    axes[1,1].set_yticks(y_pos)
    axes[1,1].set_yticklabels(top_contribucion['nombre'])

axes[1,1].set_title('Top 10 - Contribución Ofensiva', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Contribución Ofensiva (Goles + 0.7×Asistencias)', fontsize=12)
axes[1,1].set_ylabel('')
axes[1,1].grid(True, alpha=0.3, axis='x')

# Añadir valores en las barras
for i, v in enumerate(top_contribucion['contribucion_ofensiva']):
    axes[1,1].text(v + 0.2, i, f'{v:.1f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Dashboard principal generado")
print("📈 Muestra distribuciones, correlaciones y rankings clave para scouting")

---
# PARTE 4: INSIGHTS Y RECOMENDACIONES (10 puntos)

## 4.1 Análisis de Scouting

In [ ]:
# Análisis integral para responder preguntas clave de scouting
print("🎯 ANÁLISIS INTEGRAL DE SCOUTING")
print("=" * 45)

print("\n❓ PREGUNTA 1: ¿Qué perfil de jugador ofrece mejor relación calidad-precio?")
print("-" * 70)

# Crear perfiles de jugadores simplificados
def crear_perfil_simple(row):
    if row['posicion'] == 'Delantero':
        if row['goles_por_partido'] >= 0.6:
            return 'Goleador Elite'
        else:
            return 'Delantero Sólido'
    elif row['posicion'] == 'Mediocampo':
        if row['asistencias_por_partido'] >= 0.6:
            return 'Creador Elite'
        else:
            return 'Mediocampo Versátil'
    elif row['posicion'] == 'Defensa':
        return 'Defensa Sólido'
    else:
        return 'Portero'

df_jugadores['perfil_simple'] = df_jugadores.apply(crear_perfil_simple, axis=1)

# Análisis de relación calidad-precio por perfil
calidad_precio = df_jugadores.groupby('perfil_simple').agg({
    'valor_mercado_estimado': 'mean',
    'contribucion_ofensiva': 'mean',
    'jugador_id': 'count'
}).round(2)

# Calcular índice de valor (contribución/precio)
calidad_precio['indice_valor'] = (calidad_precio['contribucion_ofensiva'] / 
                                 calidad_precio['valor_mercado_estimado'] * 100000).round(3)

calidad_precio = calidad_precio.sort_values('indice_valor', ascending=False)
calidad_precio.columns = ['Valor_Promedio', 'Contribución_Prom', 'Cantidad', 'Índice_Valor']

print(calidad_precio)

mejor_perfil = calidad_precio.index[0]
print(f"\n✅ RESPUESTA: El perfil '{mejor_perfil}' ofrece la mejor relación calidad-precio")
print(f"   Índice de valor: {calidad_precio.iloc[0]['Índice_Valor']:.3f}")
print(f"   Contribución promedio: {calidad_precio.iloc[0]['Contribución_Prom']:.1f}")
print(f"   Valor promedio: ${calidad_precio.iloc[0]['Valor_Promedio']:,.0f}")

print("\n❓ PREGUNTA 2: ¿En qué posiciones hay más talento disponible?")
print("-" * 60)

# Jugadores de alto rendimiento por posición (top 25%)
umbral_alto = df_jugadores['contribucion_ofensiva'].quantile(0.75)
alto_rendimiento = df_jugadores[df_jugadores['contribucion_ofensiva'] >= umbral_alto]
talentos_por_posicion = alto_rendimiento['posicion'].value_counts()

print("Análisis de disponibilidad de talento:")
for posicion in df_jugadores['posicion'].unique():
    total = len(df_jugadores[df_jugadores['posicion'] == posicion])
    talentos = talentos_por_posicion.get(posicion, 0)
    porcentaje_talento = (talentos / total) * 100 if total > 0 else 0
    
    print(f"\n{posicion}:")
    print(f"  • Total jugadores: {total}")
    print(f"  • Talentos destacados: {talentos} ({porcentaje_talento:.1f}%)")

posicion_mas_talento = talentos_por_posicion.index[0]
print(f"\n✅ RESPUESTA: '{posicion_mas_talento}' tiene más talento disponible ({talentos_por_posicion.iloc[0]} talentos destacados)")

## 4.2 Recomendaciones Estratégicas

In [ ]:
# Recomendaciones estratégicas basadas en el análisis completo
print("📋 RECOMENDACIONES ESTRATÉGICAS PARA SCOUTING")
print("=" * 55)

print("\n🎯 1. LISTA DE 5 JUGADORES PRIORITARIOS PARA SCOUTING")
print("-" * 55)

# Seleccionar jugadores prioritarios basado en múltiples criterios
# Jugadores con excelente contribución y valor moderado
mediana_valor = df_jugadores['valor_mercado_estimado'].median()
top_contribucion = df_jugadores['contribucion_ofensiva'].quantile(0.8)

oportunidades = df_jugadores[
    (df_jugadores['valor_mercado_estimado'] <= mediana_valor * 1.2) & 
    (df_jugadores['contribucion_ofensiva'] >= top_contribucion)
].sort_values('contribucion_ofensiva', ascending=False)

# Si no hay suficientes, tomar los mejores por contribución
if len(oportunidades) < 5:
    jugadores_prioritarios = df_jugadores.nlargest(5, 'contribucion_ofensiva')
else:
    jugadores_prioritarios = oportunidades.head(5)

print("🌟 JUGADORES PRIORITARIOS RECOMENDADOS:")
for i, (idx, jugador) in enumerate(jugadores_prioritarios.iterrows(), 1):
    print(f"\n{i}. {jugador['nombre']} ({jugador['edad']} años, {jugador['posicion']})")
    print(f"   📍 Equipo: {jugador['equipo']} | Torneo: {jugador['torneo']}")
    print(f"   ⚽ Contribución: {jugador['contribucion_ofensiva']:.1f} | Valor: ${jugador['valor_mercado_estimado']:,.0f}")
    print(f"   📊 Stats: {jugador['goles']} goles, {jugador['asistencias']} asistencias en {jugador['partidos_jugados']} partidos")

print("\n\n📊 2. BENCHMARKS DE RENDIMIENTO PARA EVALUACIÓN")
print("-" * 52)

# Calcular benchmarks por posición
print("Benchmarks por posición:")
for posicion in df_jugadores['posicion'].unique():
    datos_pos = df_jugadores[df_jugadores['posicion'] == posicion]
    print(f"\n{posicion}:")
    print(f"  • Goles/partido: {datos_pos['goles_por_partido'].mean():.2f}")
    print(f"  • Asistencias/partido: {datos_pos['asistencias_por_partido'].mean():.2f}")
    print(f"  • Precisión pases: {datos_pos['precision_pases'].mean():.1f}%")
    print(f"  • Valor promedio: ${datos_pos['valor_mercado_estimado'].mean():,.0f}")

print("\n" + "=" * 55)
print("📈 RESUMEN EJECUTIVO DE RECOMENDACIONES:")
print("=" * 55)
print(f"• 5 jugadores identificados como prioritarios")
print(f"• Enfoque en {posicion_mas_talento} (mayor disponibilidad de talento)")
print(f"• Perfil '{mejor_perfil}' ofrece mejor relación calidad-precio")
print(f"• Benchmarks establecidos para evaluación continua")

---
# CONCLUSIONES Y PRÓXIMOS PASOS

## Lo que Aprendimos en el Bloque 2

¡Felicitaciones! En este caso práctico hemos aplicado exitosamente todas las técnicas avanzadas de análisis de datos del Bloque 2:

### ✅ **Exploración Avanzada de Datos:**
- Implementamos análisis exploratorio sistemático con funciones personalizadas
- Identificamos y manejamos la calidad de los datos
- Creamos variables derivadas específicas para el dominio deportivo
- Aplicamos técnicas de validación y limpieza de datos

### ✅ **Análisis Estadístico Descriptivo:**
- Realizamos análisis comparativo por categorías (posición, torneo, edad)
- Calculamos correlaciones para identificar factores predictivos
- Implementamos clustering básico para perfiles de jugadores
- Desarrollamos métricas compuestas (contribución ofensiva, índice de disciplina)

### ✅ **Visualización Profesional:**
- Creamos dashboards informativos con seaborn y matplotlib
- Implementamos mapas de calor tácticos para análisis espacial
- Desarrollamos componentes interactivos con widgets
- Aplicamos principios de diseño para comunicación efectiva

### ✅ **Aplicación de Scouting:**
- Identificamos jugadores subestimados y talentos emergentes
- Desarrollamos estrategias de reclutamiento basadas en datos
- Establecimos benchmarks de rendimiento por posición
- Creamos sistemas de alerta temprana para gestión de riesgos

## 🚀 Próximos Pasos

Con estas habilidades avanzadas de análisis de datos ya estás preparado para:
- **Bloque 3:** Modelos predictivos y machine learning aplicado al fútbol
- **Proyectos profesionales:** Análisis de scouting en organizaciones deportivas reales
- **Especialización:** Desarrollo de herramientas de análisis táctico y rendimiento

## 📊 Valor Profesional

Este proyecto demuestra competencias directamente aplicables en:
- **Análisis deportivo profesional**
- **Ciencia de datos aplicada**
- **Visualización de información compleja**
- **Toma de decisiones basada en datos**

¡Excelente trabajo aplicando técnicas avanzadas de análisis de datos al scouting deportivo! 🎉⚽📈